In [35]:
# For Dashboarding
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import numpy as np
from dash.dependencies import Input, Output,State # State is for holding the input and do a "Submit" button
import json
import base64

In [36]:
# For scarping stock data from yahoo
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import DataReader
from pandas_datareader import data as web
# For time stamps
from datetime import datetime
# For publishing the dashboard
import dash_auth 

In [37]:
# create a list of Stock options for dropdown menu
nsdq=pd.read_csv('NASDAQcompanylist.csv')
nsdq.set_index('Symbol',inplace=True)
## since the drop-down menu takes in a list of dictionary object [{label:x,value:y}]  
options=[]
for tic in nsdq.index:
    mydict={}
    mydict['label']=str(nsdq.loc[tic,'Name'])+' '+tic
    mydict['value']=tic
    options.append(mydict)

In [38]:
USERNAME_PASSWORD_PAIRS = [
    ['admin', 'password'],['wangzijian', '123456']
]

app=dash.Dash()
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)

app.layout=html.Div([
     
    html.H1('Interactive Stock Ticker Dashboard Demo'),
    html.H2('Developed by Zijian Wang'),
    html.Div([  
    html.H3('Enter a Stock Symbol(eg: AAPL):',style={'paddingRight':'30px'}),
    dcc.Dropdown(id='my_ticker_symbol',options=options,value=['TSLA'],multi=True)],
        style={'display':'inline-block','verticalAlign':'top','width':'30%'}),
    
    html.Div([html.H3('Select a Start and End Date:'),
             dcc.DatePickerRange(id='my_date_picker',
                                min_date_allowed=datetime(2015,1,1),
                                max_date_allowed=datetime.today(),
                                start_date=datetime(2018,1,1),
                                end_date=datetime.today())],
            style={'display':'inline-block'}),
    
    html.Div([
        
        html.Button(id='submit-button',n_clicks=0,children='Submit',
                   style={'fontSize':24,'marginLeft':'30px'})
    ],style={'display':'inline-block'}),
    
    dcc.Graph(id='my_graph',figure={  
        "data":[{'x':[1,2],'y':[3,1]}]
    ,'layout':{'title':'Default Title'}})        
        
    ])
    
@app.callback(Output('my_graph','figure'),
              [Input('submit-button','n_clicks')],
             [State('my_ticker_symbol','value'),
             State('my_date_picker','start_date'),
             State('my_date_picker','end_date')])
# each updated graphs or objects needs a function to do the updating
def update_graph(n_clicks,stock_ticker,start_date,end_date):
    start=datetime.strptime(start_date[:10],'%Y-%m-%d') # convert the string to datetime object
    end=datetime.strptime(end_date[:10],'%Y-%m-%d')
    
    traces=[]
    for tic in stock_ticker:
        df=web.DataReader(tic,'iex',start,end)
        traces.append({'x':df.index,'y':df['close'],'name':tic})
    
    fig={"data":traces,
        'layout':{'title':stock_ticker}}
    return fig

if __name__=='__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2018 17:00:47] "GET / HTTP/1.1" 401 -
127.0.0.1 - - [13/Nov/2018 17:00:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 17:00:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 17:00:53] "GET /_dash-dependencies HTTP/1.1" 200 -


1y


127.0.0.1 - - [13/Nov/2018 17:00:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 17:00:53] "GET /favicon.ico HTTP/1.1" 200 -
